In [7]:
import tensorflow as tf
import numpy as np
import matplotlib as pyplot
tf.__version__
np.__version__

'1.15.0'

In [11]:
bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)
def pullBandit(bandit):
    #Get a random number.
    result = np.random.randn(1)
    if result > bandit:
        #return a positive reward.
        return 1
    else:
        #return a negative reward.
        return -1

In [25]:
tf.reset_default_graph()
# init = tf.global_variables_initializer()
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)
print(weights,chosen_action,action_holder ,responsible_weight,loss )

<tf.Variable 'Variable:0' shape=(4,) dtype=float32_ref> Tensor("ArgMax:0", shape=(), dtype=int64) Tensor("Placeholder_1:0", shape=(1,), dtype=int32) Tensor("Slice:0", shape=(1,), dtype=float32) Tensor("Neg:0", shape=(1,), dtype=float32)


In [27]:
total_episodes = 1000 #Set total number of episodes to train agent on.
total_reward = np.zeros(num_bandits) #Set scoreboard for bandits to 0.
e = 0.1 #Set the chance of taking a random action.

init = tf.initialize_all_variables()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        #Choose either a random action or one from our network.
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
        reward = pullBandit(bandits[action]) #Get our reward from picking one of the bandits.
        
        #Update the network.
        _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #Update our running tally of scores.
        total_reward[action] += reward
        if i % 50 == 0:
            print ("Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
        i+=1
print ("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print ("...and it was right!")
else:
    print ("...and it was wrong!")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Running reward for the 4 bandits: [-1.  0.  0.  0.]
Running reward for the 4 bandits: [-1. -2.  0. 42.]
Running reward for the 4 bandits: [-2. -3. -1. 89.]
Running reward for the 4 bandits: [ -1.  -1.   0. 135.]
Running reward for the 4 bandits: [ -1.  -2.  -1. 181.]
Running reward for the 4 bandits: [ -3.  -1.  -1. 228.]
Running reward for the 4 bandits: [ -4.  -3.  -1. 273.]
Running reward for the 4 bandits: [ -4.  -3.   0. 318.]
Running reward for the 4 bandits: [ -4.  -3.   1. 367.]
Running reward for the 4 bandits: [ -3.  -3.   1. 412.]
Running reward for the 4 bandits: [ -4.  -1.   1. 459.]
Running reward for the 4 bandits: [ -5.   0.   0. 504.]
Running reward for the 4 bandits: [ -5.   0.   0. 552.]
Running reward for the 4 bandits: [ -3.   1.  -1. 598.]
Running reward for the 4 bandits: [ -3.   0.  -1. 647.]
Running reward for the 4 bandits: [ -1.   0.   0. 692.]
Running reward for the 4 bandits: [ -3.   